# Introduction

Notebook para fazer download de datasets de TFDS (Tensorflow Datasets).

Datasets baixados:
* Celeb A

# Import Libraries

In [1]:
import os
import cv2 
import shutil
import sklearn

import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

2022-07-20 20:34:54.099011: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


# Datasets Root Directory Path

In [2]:
DATASETS_ROOT_DIR = '/home/guilherme/data1/Dropbox/Link to Desktop/Doutorado/Datasets/'

# Utility Functions

In [3]:
def format_celeba_split(ds):
    df = tfds.as_dataframe(ds)
    df_labels = pd.get_dummies(df.label)
    df = pd.concat([df, df_labels], axis=1)
    #df = df.drop(columns=['label'], inplace=False)
    #df = df.rename(columns={x: f'n_{x}' for x in df.columns if type(x) is int}, inplace=False)
    #df['img_name'] = [f'image_{x}' for x in range(ds.cardinality())]
    return df


def record_dataset(df, ds_split, split_name, ds_name):
    dir_path = os.path.join(f'{DATASETS_ROOT_DIR}', ds_name, split_name)
    shutil.rmtree(dir_path, ignore_errors=True)
    os.makedirs(dir_path, exist_ok=True)
    
    for idx,row in df.iterrows():        
        img_path = os.path.join(dir_path, row.img_name + '.jpg')
        cv2.imwrite(img_path, row.image)
        df.loc[idx, 'img_name'] = img_path
    
    split_labels_df = df[['img_name']+[f'n_{x}' for x in range(10)]]
    
    labels_dir_path = os.path.join(DATASETS_ROOT_DIR, ds_name)
    os.makedirs(labels_dir_path, exist_ok=True)
    
    labels_file_path = os.path.join(labels_dir_path, split_name + '_data.csv')
    split_labels_df.to_csv(labels_file_path, index=False)
                    

# Celeb A Dataset

In [5]:
[ds_train, ds_valid, ds_test] = tfds.load('celeb_a', split=['train','validation','test'], shuffle_files=True)

#df_train = format_celeba_split(ds_train)
#df_valid = format_celeba_split(ds_valid)
#df_test = format_celeba_split(ds_test)
  
#display(df_train.head())
#display(df_valid.head())
#display(df_test.head())

NonMatchingChecksumError: Artifact https://drive.google.com/uc?export=download&id=0B7EVK8r0v71pZjFTYXZWM3FlRnM, downloaded to /home/guilherme/tensorflow_datasets/downloads/ucexport_download_id_0B7EVK8r0v71pZjFTYXZWM3FlDDaXUAQO8EGH_a7VqGNLRtW52mva1LzDrb-V723OQN8.tmp.9946b8bf9fe2426ba1cc43ad67bf1f58/uc, has wrong checksum. Expected: UrlInfo(size=1.34 GiB, checksum='46fb89443c578308acf364d7d379fe1b9efb793042c0af734b6112e4fd3a8c74', filename='img_align_celeba.zip'). Got: UrlInfo(size=1.95 KiB, checksum='32bab9118c8cc2f2a41708170718474fa01c488f76493b77da0f52755f9d63e2', filename='uc').To debug, see: https://www.tensorflow.org/datasets/overview#fixing_nonmatchingchecksumerror

## Record Images Files

In [5]:
record_dataset(df_train, ds_train, 'train', 'fashion_mnist')
record_dataset(df_valid, ds_valid, 'valid', 'fashion_mnist')
record_dataset(df_test, ds_test, 'test', 'fashion_mnist')